In [1]:
from copy import copy
import json
import pandas as pd
import textwrap

# BTE Sleeve

Some queries result in an explosion of the number of results. This is bad for performance of the server and also not very useful to the user. This notebook explores using PFOCR as a "compression sleeve" to dampen these types of explosions. I don't have a CC-licensed image of a compression sleeve, so instead here's an image of a heat-shrink tube.

<a title="oomlout, CC BY-SA 2.0 &lt;https://creativecommons.org/licenses/by-sa/2.0&gt;, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:HESH-06-L-STAN-01_(9601242883).jpg"><img width="512" alt="HESH-06-L-STAN-01 (9601242883)" src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d3/HESH-06-L-STAN-01_%289601242883%29.jpg/512px-HESH-06-L-STAN-01_%289601242883%29.jpg"></a>

Let's try BTE Sleeve on the test query [A.2_RHOBTB2_twohop](https://github.com/NCATSTranslator/minihackathons/blob/main/2021-12_demo/workflowA/A.2_RHOBTB2_twohop.json):

`RHOBTB2 Gene`--entity_regulates_entity|genetically_interacts_with--`Any Gene`--related_to--`Any SmallMolecule`

`n0`--e01--`n1`--e02--`n2`


Note BTE Sleeve currently only works on queries with nodes of categories supported by PFOCR, e.g., `biolink:Gene`, `biolink:SmallMolecule`, `biolink:Disease`. TODO: update the Fisher's Exact test to work with node categories not in PFOCR.

Also, BTE Sleeve currently assumes `n0` has an `ids` parameter. TODO: update to handle `ids` params on any q_node.

In [2]:
import requests
import requests_cache


requests_cache.install_cache("pfocr_cache", allowable_methods=['GET', 'POST'])

## Get BTE TRAPI Results

In [3]:
query = {
    "message": {
        "query_graph": {
            "edges": {
                "e01": {
                    "object": "n0",
                    "subject": "n1",
                    "predicates": [
                        "biolink:entity_regulates_entity",
                        "biolink:genetically_interacts_with"
                    ]
                },
                "e02": {
                    "object": "n1",
                    "subject": "n2",
                    "predicates": [
                        "biolink:related_to"
                    ]
                }
            },
            "nodes": {
                "n0": {
                    "ids": [
                        "NCBIGene:5872"
                    ],
                    "categories": [
                        "biolink:Gene"
                    ]
                },
                "n1": {
                    "categories": [
                        "biolink:Gene"
                    ],
                },
                "n2": {
                    "categories": [
                        "biolink:SmallMolecule"
                    ]
                }
            }
        }
    }
}

trapi_response = requests.post("http://localhost:3000/v1/query", json=query)
#trapi_response = requests.post("https://api.bte.ncats.io/v1/query", json=query)
#trapi_response = requests.post("https://arax.ncats.io/api/rtxkg2/v1.2/query", json=query)
#trapi_response = requests.post("https://arax.ncats.io/api/arax/v1.2/query", json=query)
print(trapi_response.status_code)
if trapi_response.status_code != 200:
    print(trapi_response.text)
if trapi_response.from_cache:
    print(f"got response from cache")

trapi_message = trapi_response.json()["message"]

#print(json.dumps(query))

200
got response from cache


In [4]:
# note we're also including any categories from the results
curie_categories = set()
for v in  query["message"]["query_graph"]["nodes"].values():
    if "categories" in v:
        for category in v["categories"]:
            curie_categories.add(category)

In [5]:
# for genes/gene products, chemicals and diseases
preferred_prefixes = set(["NCBIGene", "MESH"])
unified_names = set()
unified_curies = set()
unification_failed_curies = set()
unified_prefixes = set()
all_prefixes = set()
curie_to_name = dict()
curie_to_unified_curie = dict()
for k, v in trapi_message["knowledge_graph"]["nodes"].items():
    name = v["name"]
    for category in v["categories"]:
        curie_categories.add(category)
        
    for a in v["attributes"]:
        if a["attribute_type_id"] == "biolink:xref":
            curies = a["value"]
            
            # k should always be one of the curies
            if not k in curies:
                raise Exception(f"key {k} not in {curies}")
            
            unified_curie = None
            intersecting_unified_curies = unified_curies.intersection(set(curies))
            if len(intersecting_unified_curies) > 1:
                raise Exception(f"matching multiple: {k} to {list(intersecting_unified_curies)}")
            elif len(intersecting_unified_curies) == 1:
                unified_curie = list(intersecting_unified_curies)[0]
            else:
                # get curie for preferred prefix. usually this is k, but not always.
                for curie in curies:
                    [prefix, identifier] = curie.split(":")
                    if prefix in preferred_prefixes:
                        unified_curie = curie
                        unified_curies.add(unified_curie)
                        break
                        
            if not unified_curie:
                if k in curie_to_unified_curie:
                    unified_curie = curie_to_unified_curie[k]
                else:
                    unification_failed_curies.add(k)
                    break
                    
                #raise Exception(f"failed to find a unified curie for {k} in {curies}")
                
            [unified_prefix, unified_identifier] = unified_curie.split(":")
            unified_prefixes.add(unified_prefix)
            
            for curie in curies:
                [prefix, identifier] = curie.split(":")
                all_prefixes.add(prefix)
                if not curie in curie_to_unified_curie:
                    curie_to_unified_curie[curie] = unified_curie
                if not curie in curie_to_name:
                    curie_to_name[curie] = name
                    unified_names.add(name)
                elif curie_to_name[curie] != name:
                    print(f"curie {curie} has multiple primary names: {curie_to_name[curie]} and {name}")
                    #raise Exception(f"curie{curie} has multiple names: {curie_to_name[curie]} and {name}")


print(f'curie_categories: {curie_categories}')
print("")
print(f"curie_to_name key count: {len(curie_to_name.keys())}")
print(f"curie_to_unified_curie key count: {len(curie_to_unified_curie.keys())}")
print("")
print("all curie prefixes found:")
print(all_prefixes)
print("")
print("unified curie prefixes found:")
print(unified_prefixes)
print("")
print(f"failed to unify {len(unification_failed_curies)} curies")

curie UMLS:C0903898 has multiple primary names: NATEGLINIDE and nateglinide
curie UMLS:C0021641 has multiple primary names: Iletin and Insulin
curie_categories: {'biolink:MolecularMixture', 'biolink:ChemicalEntity', 'biolink:ComplexMolecularMixture', 'biolink:Protein', 'biolink:SmallMolecule', 'biolink:PhenotypicFeature', 'biolink:Gene', 'biolink:Polypeptide'}

curie_to_name key count: 22097
curie_to_unified_curie key count: 22097

all curie prefixes found:
{'CHEMBL.COMPOUND', 'KEGG.COMPOUND', 'UniProtKB', 'MESH', 'PR', 'PUBCHEM.COMPOUND', 'HGNC', 'DRUGBANK', 'NCBIGene', 'INCHIKEY', 'GTOPDB', 'OMIM', 'HMDB', 'UNII', 'UMLS', 'CHEBI', 'CAS', 'ENSEMBL', 'DrugCentral'}

unified curie prefixes found:
{'MESH', 'NCBIGene'}

failed to unify 1007 curies


In [6]:
columns = []
q_node_id_keys = set(["object", "subject"])
q_node_ids = []
q_edge_ids = []
for q_edge_id,edge_v in query["message"]["query_graph"]["edges"].items():
    q_edge_ids.append(q_edge_id)
    q_node_id_found = False
    for k,v in edge_v.items():
        if (k in q_node_id_keys) and (type(v) is str):
            if v not in columns:
                q_node_ids.append(v)
                columns.append(v)
            if not q_node_id_found:
                q_node_id_found = True
                columns.append(q_edge_id)
                
for q_node_id in q_node_ids:
    columns.append(f"{q_node_id}_curie")
    columns.append(f"{q_node_id}_unified_curie")
    
columns.append("trapi_result_curie_combo")
    
print(columns)
print(q_node_ids)
print(q_edge_ids)

['n0', 'e01', 'n1', 'e02', 'n2', 'n0_curie', 'n0_unified_curie', 'n1_curie', 'n1_unified_curie', 'n2_curie', 'n2_unified_curie', 'trapi_result_curie_combo']
['n0', 'n1', 'n2']
['e01', 'e02']


In [7]:
trapi_result_columns = []
for i in range(len(q_node_ids)):
    trapi_result_columns.append(q_node_ids[i])
    if i < len(q_edge_ids):
        trapi_result_columns.append(q_edge_ids[i])
print(trapi_result_columns)

['n0', 'e01', 'n1', 'e02', 'n2']


In [8]:
from copy import deepcopy


trapi_results = trapi_message["results"]

result_row_data = []
for trapi_result in trapi_results:
    curie_to_qnode_ids = dict()
    for qnode_id, entries in trapi_result["node_bindings"].items():
        for entry in entries:
            curie = entry["id"]
            if curie not in curie_to_qnode_ids:
                curie_to_qnode_ids[curie] = []
            curie_to_qnode_ids[curie].append(qnode_id)
        
    row_data_template = dict()
    q_edge_id_to_predicates = dict()
    trapi_result_curies = set()
    for qedge_id, entries in trapi_result["edge_bindings"].items():
        for entry in entries:
            curie = entry["id"]
            kg_entry = trapi_message["knowledge_graph"]["edges"][curie]
            subject_curie = kg_entry["subject"]
            object_curie = kg_entry["object"]
            predicate_curie = kg_entry["predicate"]
            [predicate_prefix, predicate_identifier] = predicate_curie.split(":")
            
            if qedge_id not in q_edge_id_to_predicates:
                q_edge_id_to_predicates[qedge_id] = set()
            q_edge_id_to_predicates[qedge_id].add(predicate_identifier)

            for curie in [subject_curie, object_curie]:
                for qnode_id in curie_to_qnode_ids[curie]:
                    if curie in curie_to_unified_curie:
                        unified_curie = curie_to_unified_curie[curie]
                    else:
                        break

                    name = curie_to_name[curie]
                    row_data_template[qnode_id] = name

                    trapi_result_curies.add(unified_curie)
                    row_data_template[qnode_id + "_curie"] = curie
                    row_data_template[qnode_id + "_unified_curie"] = unified_curie
                    
    if len(trapi_result_curies) != len(q_node_ids):
        #print(f'skipping {list(curie_to_qnode_ids.keys())}')
        continue
        
    row_data_template["trapi_result_curie_combo"] = tuple(sorted(trapi_result_curies))
    q_edge_ids_processed = set()
    row_datas = [row_data_template]
    for q_edge_id,predicates in q_edge_id_to_predicates.items():
        next_row_datas = []
        for row_data in row_datas:
            for predicate in predicates:
                next_row_data = deepcopy(row_data)
                next_row_data[q_edge_id] = predicate
                next_row_datas.append(
                    next_row_data 
                )
        row_datas = next_row_datas
    result_row_data += row_datas
        
print("warning: predicate direction(s) may be switched")
results_df = pd.DataFrame.from_records(result_row_data, columns=columns).drop_duplicates()
results_df

,n0,e01,n1,e02,n2,n0_curie,n0_unified_curie,n1_curie,n1_unified_curie,n2_curie,n2_unified_curie,trapi_result_curie_combo
0,RAB13,entity_positively_regulates_entity,MST1,entity_negatively_regulates_entity,Chloride ion,NCBIGene:5872,NCBIGene:5872,NCBIGene:4485,NCBIGene:4485,PUBCHEM.COMPOUND:312,MESH:D002712,"(MESH:D002712, NCBIGene:4485, NCBIGene:5872)"
1,RAB13,entity_positively_regulates_entity,MST1,sensitivity_associated_with,Cisplatin,NCBIGene:5872,NCBIGene:5872,NCBIGene:4485,NCBIGene:4485,PUBCHEM.COMPOUND:5702198,MESH:C034697,"(MESH:C034697, NCBIGene:4485, NCBIGene:5872)"
2,RAB13,entity_positively_regulates_entity,MST1,resistance_associated_with,Cisplatin,NCBIGene:5872,NCBIGene:5872,NCBIGene:4485,NCBIGene:4485,PUBCHEM.COMPOUND:5702198,MESH:C034697,"(MESH:C034697, NCBIGene:4485, NCBIGene:5872)"
3,RAB13,entity_positively_regulates_entity,MST1,entity_positively_regulates_entity,Cisplatin,NCBIGene:5872,NCBIGene:5872,NCBIGene:4485,NCBIGene:4485,PUBCHEM.COMPOUND:5702198,MESH:C034697,"(MESH:C034697, NCBIGene:4485, NCBIGene:5872)"
4,RAB13,entity_positively_regulates_entity,MST1,entity_negatively_regulates_entity,liraglutide,NCBIGene:5872,NCBIGene:5872,NCBIGene:4485,NCBIGene:4485,DrugCentral:4164,MESH:D000069450,"(MESH:D000069450, NCBIGene:4485, NCBIGene:5872)"
...,...,...,...,...,...,...,...,...,...,...,...,...
7273,RAB13,entity_regulates_entity,PAF1,resistance_associated_with,2-(2-Chloro-4-iodophenylamino)-N-(cyclopropylm...,NCBIGene:5872,NCBIGene:5872,NCBIGene:54623,NCBIGene:54623,PUBCHEM.COMPOUND:6918454,MESH:C120227,"(MESH:C120227, NCBIGene:54623, NCBIGene:5872)"
7274,RAB13,entity_regulates_entity,PAF1,resistance_associated_with,"N-[(2R)-2,3-Dihydroxypropoxy]-3,4-difluoro-2-[...",NCBIGene:5872,NCBIGene:5872,NCBIGene:54623,NCBIGene:54623,PUBCHEM.COMPOUND:9826528,MESH:C506614,"(MESH:C506614, NCBIGene:54623, NCBIGene:5872)"
7275,RAB13,entity_regulates_entity,PAF1,resistance_associated_with,"4-[4-(2,3-Dihydro-1,4-benzodioxin-6-YL)-3-meth...",NCBIGene:5872,NCBIGene:5872,NCBIGene:54623,NCBIGene:54623,PUBCHEM.COMPOUND:984170,MESH:C506244,"(MESH:C506244, NCBIGene:54623, NCBIGene:5872)"
7276,RAB13,entity_regulates_entity,PAF1,coexists_with,Formoterol,NCBIGene:5872,NCBIGene:5872,NCBIGene:54623,NCBIGene:54623,PUBCHEM.COMPOUND:3410,MESH:D000068759,"(MESH:D000068759, NCBIGene:54623, NCBIGene:5872)"


In [9]:
for q_node_id in q_node_ids:
    print(len(set(results_df[q_node_id])))

1
18
1745


In [10]:
len(set(results_df["trapi_result_curie_combo"]))

5019

Total results count (excluding any that couldn't be unified): 3,738. That's too many for a researcher to efficiently go through manually, so let's try filtering to make that more manageable.

## Match up BTE TRAPI Results & PFOCR

We're going to try using PFOCR to filter and prioritize the results.

First we need to get the PFOCR Data. We could get it from the API, but for now, we'll just go ahead and download the entire JSON file we gave to BTE.

In [11]:
pfocr_url = "https://www.dropbox.com/s/1f14t5zaseocyg6/bte_chemicals_diseases_genes.ndjson?dl=1"
pfocr_request = requests.get(pfocr_url)
print(f"status_code: {pfocr_request.status_code}")
if pfocr_request.status_code != 200:
    print(pfocr_request.text)

status_code: 200


In [12]:
curies_to_figure_ids = {}
figure_id_to_curies = {}
figure_id_to_pfocr_result = {}
for line in pfocr_request.text.splitlines():
    pfocr_result = json.loads(line)
    figure_id = pfocr_result["_id"]

    curies = set()
    for identifier in pfocr_result["associatedWith"]["mentions"]["chemicals"]["mesh"]:
        curie = "MESH:" + identifier
        if curie in curie_to_unified_curie:
            unified_curie = curie_to_unified_curie[curie]
            curies.add(unified_curie)
        else:
            #print(f"{curie} not in list")
            # this curie isn't in the BTE results, but we'll add an item for
            # the purpose of the denominator in jaccard/containment cals
            curies.add(curie)
    for identifier in pfocr_result["associatedWith"]["mentions"]["diseases"]["mesh"]:
        curie = "MESH:" + identifier
        if curie in curie_to_unified_curie:
            unified_curie = curie_to_unified_curie[curie]
            curies.add(unified_curie)
        else:
            #print(f"{curie} not in list")
            # this curie isn't in the BTE results, but we'll add an item for
            # the purpose of the denominator in jaccard/containment cals
            curies.add(curie)
    for identifier in pfocr_result["associatedWith"]["mentions"]["genes"]["ncbigene"]:
        curie = "NCBIGene:" + identifier
        if curie in curie_to_unified_curie:
            unified_curie = curie_to_unified_curie[curie]
            curies.add(unified_curie)
        else:
            #print(f"{curie} not in list")
            # this curie isn't in the BTE results, but we'll add an item for
            # the purpose of the denominator in jaccard/containment cals
            curies.add(curie)

    figure_id_to_pfocr_result[figure_id] = pfocr_result
    figure_id_to_curies[figure_id] = curies

    curies_key = tuple(sorted(curies))
    if curies_key not in curies_to_figure_ids:
        curies_to_figure_ids[curies_key] = []
    curies_to_figure_ids[curies_key].append(figure_id)

In [13]:
from SetSimilaritySearch import SearchIndex

# the reference sets are supposed to be from PFOCR only
reference_sets = list()
for curies in set(curies_to_figure_ids.keys()):
    reference_sets.append(set(curies))
# but in order to calculate the scores correctly,
# the SetSimilaritySearch library requires that
# every curie from the bte results need to be
# mentioned at least once in the reference set.
# That's the only reason we add them below, as
# one large set.

unified_curie_columns = [
    q_node_id + "_unified_curie" for q_node_id in q_node_ids
]

trapi_result_curie_combos = set()
for i, df in results_df[unified_curie_columns].drop_duplicates().iterrows():
    trapi_result_curies = []
    for unified_curie_column in unified_curie_columns:
        trapi_result_curies.append(
            df[unified_curie_column]
        )
    trapi_result_curie_combos.add(tuple(sorted(
        trapi_result_curies
    )))
    
reference_set = set()
for trapi_result_curie_combo in trapi_result_curie_combos:
    reference_set |= set(trapi_result_curie_combo)
reference_sets.append(reference_set)

matchable_node_min = 2
matchable_node_count = len(q_node_ids)
index = SearchIndex(reference_sets, similarity_func_name="containment", 
    similarity_threshold=matchable_node_min/matchable_node_count)

trapi_figure_overlap_rows = []
for trapi_result_curie_combo in trapi_result_curie_combos:
    trapi_curies = set(trapi_result_curie_combo)
    results = index.query(trapi_curies)
    for result in results:
        figure_curie_combos = reference_sets[result[0]]
        figure_curie_key = tuple(sorted(figure_curie_combos))
        
        # needed to not match bte results to themselves
        if figure_curie_key in curies_to_figure_ids:
            figure_ids = curies_to_figure_ids[figure_curie_key]
        else:
            continue
            
        score = result[1]
        
        common_curies = trapi_curies.intersection(figure_curie_combos)
        for figure_id in figure_ids:
            pfocr_result = figure_id_to_pfocr_result[figure_id]
            trapi_figure_overlap_rows.append({
                "figure_id": figure_id,
                "figure_url": pfocr_result["associatedWith"]["figureUrl"],
                "figure_title": pfocr_result["associatedWith"]["title"],
                "trapi_result_curie_combo": trapi_result_curie_combo,
                "figure_curie_combo": figure_curie_key,
                "overlap_curie_combo": tuple(sorted(common_curies)),
                "score": score,
            })


trapi_figure_overlap_df = pd.DataFrame.from_records(trapi_figure_overlap_rows)
trapi_figure_overlap_df

,figure_id,figure_url,figure_title,trapi_result_curie_combo,figure_curie_combo,overlap_curie_combo,score
0,PMC6770969__genes-10-00728-g006.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,KEGG AMPK signaling,"(MESH:D007455, NCBIGene:3630, NCBIGene:5872)","(MESH:C031143, MESH:C504504, MESH:D000105, MES...","(NCBIGene:3630, NCBIGene:5872)",0.666667
1,PMC7884075__elife-58615-fig4.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,GWAS hits in the IGF-1 pathway,"(MESH:D007455, NCBIGene:3630, NCBIGene:5872)","(MESH:D009584, NCBIGene:10000, NCBIGene:10890,...","(NCBIGene:3630, NCBIGene:5872)",0.666667
2,PMC5518130__12864_2017_3907_Fig4_HTML.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,Differently expressed protein-coding genes and...,"(MESH:D007455, NCBIGene:3630, NCBIGene:5872)","(MESH:C031143, MESH:D000105, MESH:D000214, MES...","(NCBIGene:3630, NCBIGene:5872)",0.666667
3,PMC6406872__cells-08-00192-g003.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,Structural hotspots and signaling pathways tha...,"(MESH:D007455, NCBIGene:3630, NCBIGene:5872)","(NCBIGene:1000, NCBIGene:10000, NCBIGene:10050...","(NCBIGene:3630, NCBIGene:5872)",0.666667
4,PMC5546324__nihms866088f6.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,AKT/PKB Signaling: Navigating the Network,"(MESH:D007455, NCBIGene:3630, NCBIGene:5872)","(MESH:D000249, MESH:D005947, NCBIGene:10000, N...","(NCBIGene:3630, NCBIGene:5872)",0.666667
...,...,...,...,...,...,...,...
43022,PMC7097195__41573_2019_36_Fig2_HTML.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,Lysosomes as a therapeutic target,"(MESH:C095105, NCBIGene:201475, NCBIGene:5872)","(MESH:C057620, MESH:C116960, MESH:C118025, MES...","(NCBIGene:201475, NCBIGene:5872)",0.666667
43023,PMC3805607__pone.0077489.g003.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,Comparative Transcriptional Profiling of 3 Mur...,"(MESH:C095105, NCBIGene:201475, NCBIGene:5872)","(MESH:C063665, MESH:C076555, MESH:C099410, MES...","(NCBIGene:201475, NCBIGene:5872)",0.666667
43024,PMC5851872__nihms944231f1.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,Molecular Regulation of Follicle-Stimulating H...,"(MESH:C095105, NCBIGene:201475, NCBIGene:5872)","(MESH:C554042, MESH:D004967, MESH:D011374, MES...","(NCBIGene:201475, NCBIGene:5872)",0.666667
43025,PMC7607850__12885_2020_7568_Fig2_HTML.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,ZA inhibits mevalonate pathway,"(MESH:C095105, NCBIGene:201475, NCBIGene:5872)","(MESH:C008047, MESH:C039834, MESH:D004246, MES...","(NCBIGene:201475, NCBIGene:5872)",0.666667


In [14]:
print(len(set(
    trapi_figure_overlap_df[trapi_figure_overlap_df["overlap_curie_combo"].map(len) >= 2]["trapi_result_curie_combo"]
)))
print(len(set(
    trapi_figure_overlap_df[trapi_figure_overlap_df["overlap_curie_combo"].map(len) >= 3]["trapi_result_curie_combo"]
)))
print(len(set(
    trapi_figure_overlap_df[trapi_figure_overlap_df["overlap_curie_combo"].map(len) >= 4]["trapi_result_curie_combo"]
)))
print(len(set(trapi_figure_overlap_df["figure_id"])))

3516
33
0
2906


### trapi_curie_combo_count_by_figure_df

In [15]:
trapi_curie_combo_count_by_figure_df = trapi_figure_overlap_df[["figure_id", "trapi_result_curie_combo"]].rename(columns={
    "trapi_result_curie_combo": "trapi_curie_combo_count"
}).groupby("figure_id").count().sort_values(
    by="trapi_curie_combo_count", ascending=False)
trapi_curie_combo_count_by_figure_df

,trapi_curie_combo_count
figure_id,
PMC6406872__cells-08-00192-g003.jpg,1321
PMC6472122__MMR-19-05-3564-g04.jpg,1058
PMC4342183__dddt-9-1103Fig6.jpg,1048
PMC5518130__12864_2017_3907_Fig4_HTML.jpg,1046
PMC6770969__genes-10-00728-g006.jpg,1031
...,...
PMC4709637__srep19226-f1.jpg,1
PMC4710172__nihms616784f2.jpg,1
PMC4710172__nihms616784f3.jpg,1


### figure count/score by trapi curie combo

In [16]:
trapi_figure_overlap_df[["trapi_result_curie_combo", "figure_id"]].rename(columns={
    "figure_id": "figure_count"
}).groupby("trapi_result_curie_combo").count().sort_values(
    by="figure_count", ascending=False).head(10)

,figure_count
trapi_result_curie_combo,
"(NCBIGene:4088, NCBIGene:4089, NCBIGene:5872)",1790
"(MESH:D002118, NCBIGene:4775, NCBIGene:5872)",395
"(MESH:C531549, NCBIGene:201475, NCBIGene:5872)",190
"(MESH:C527797, NCBIGene:201475, NCBIGene:5872)",190
"(MESH:C528327, NCBIGene:201475, NCBIGene:5872)",190
"(MESH:C528328, NCBIGene:201475, NCBIGene:5872)",190
"(MESH:C528561, NCBIGene:201475, NCBIGene:5872)",190
"(MESH:C401859, NCBIGene:201475, NCBIGene:5872)",190
"(MESH:D014212, NCBIGene:201475, NCBIGene:5872)",190


In [17]:
trapi_result_figure_score_df = trapi_figure_overlap_df[["trapi_result_curie_combo", "score"]].rename(
    columns={"score": "cumulative_trapi_result_curie_combo_figure_score"}
).groupby("trapi_result_curie_combo").sum().sort_values(
    by="cumulative_trapi_result_curie_combo_figure_score", ascending=False)
trapi_result_figure_score_df.head(10)

,cumulative_trapi_result_curie_combo_figure_score
trapi_result_curie_combo,
"(NCBIGene:4088, NCBIGene:4089, NCBIGene:5872)",1195.333333
"(MESH:D002118, NCBIGene:4775, NCBIGene:5872)",263.333333
"(MESH:D019821, NCBIGene:201475, NCBIGene:5872)",131.000000
"(MESH:C031143, NCBIGene:201475, NCBIGene:5872)",127.666667
"(MESH:C085911, NCBIGene:201475, NCBIGene:5872)",127.000000
"(MESH:C546624, NCBIGene:201475, NCBIGene:5872)",127.000000
"(MESH:D020123, NCBIGene:201475, NCBIGene:5872)",127.000000
"(MESH:D008985, NCBIGene:201475, NCBIGene:5872)",127.000000
"(MESH:D000068196, NCBIGene:201475, NCBIGene:5872)",127.000000


### TODO: why is NCBIGene:5111 returned as a SmallMolecule?

In [18]:
results_df[results_df["trapi_result_curie_combo"] == ("NCBIGene:23221", "NCBIGene:5111", "NCBIGene:595")]

,n0,e01,n1,e02,n2,n0_curie,n0_unified_curie,n1_curie,n1_unified_curie,n2_curie,n2_unified_curie,trapi_result_curie_combo


In [19]:
results_with_figures_df = trapi_figure_overlap_df.merge(
    results_df,
    on="trapi_result_curie_combo",
    how="left").merge(
    trapi_result_figure_score_df.reset_index(),
    on="trapi_result_curie_combo",
    how="left",
    validate="many_to_one"
).merge(
    trapi_curie_combo_count_by_figure_df.reset_index().rename(columns={
        "trapi_curie_combo_count": "trapi_curie_combo_count_by_figure"
    }),
    on="figure_id",
    how="left",
    validate="many_to_one"
).sort_values(
    by="cumulative_trapi_result_curie_combo_figure_score", ascending=False
)

results_with_figures_df

,figure_id,figure_url,figure_title,trapi_result_curie_combo,figure_curie_combo,overlap_curie_combo,score,n0,e01,n1,e02,n2,n0_curie,n0_unified_curie,n1_curie,n1_unified_curie,n2_curie,n2_unified_curie,cumulative_trapi_result_curie_combo_figure_score,trapi_curie_combo_count_by_figure
77770,PMC1998874__335_2007_9040_Fig4_HTML.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,Genes comprising simple linearly ordered pathw...,"(NCBIGene:4088, NCBIGene:4089, NCBIGene:5872)","(NCBIGene:4086, NCBIGene:4087, NCBIGene:4088, ...","(NCBIGene:4088, NCBIGene:4089)",0.666667,RAB13,genetically_interacts_with,SMAD4,part_of,SMAD3,NCBIGene:5872,NCBIGene:5872,NCBIGene:4089,NCBIGene:4089,NCBIGene:4088,NCBIGene:4088,1195.333333,1
78846,PMC5765207__18_2017_2608_Fig3_HTML.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,Developmentally regulated signaling pathways i...,"(NCBIGene:4088, NCBIGene:4089, NCBIGene:5872)","(MESH:D014443, NCBIGene:10000, NCBIGene:11211,...","(NCBIGene:4088, NCBIGene:4089)",0.666667,RAB13,genetically_interacts_with,SMAD4,part_of,SMAD3,NCBIGene:5872,NCBIGene:5872,NCBIGene:4089,NCBIGene:4089,NCBIGene:4088,NCBIGene:4088,1195.333333,2
78844,PMC8038106__ijms-22-03471-g001.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8...,Molecular pathophysiology of hereditary hemorr...,"(NCBIGene:4088, NCBIGene:4089, NCBIGene:5872)","(MESH:C064206, MESH:D011692, NCBIGene:2022, NC...","(NCBIGene:4088, NCBIGene:4089)",0.666667,RAB13,genetically_interacts_with,SMAD4,part_of,SMAD3,NCBIGene:5872,NCBIGene:5872,NCBIGene:4089,NCBIGene:4089,NCBIGene:4088,NCBIGene:4088,1195.333333,1
78843,PMC548008__zmb0040547470007.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,The molecular mechanisms involved in the TGF-B...,"(NCBIGene:4088, NCBIGene:4089, NCBIGene:5872)","(NCBIGene:10000, NCBIGene:1432, NCBIGene:14685...","(NCBIGene:4088, NCBIGene:4089)",0.666667,RAB13,genetically_interacts_with,SMAD4,part_of,SMAD3,NCBIGene:5872,NCBIGene:5872,NCBIGene:4089,NCBIGene:4089,NCBIGene:4088,NCBIGene:4088,1195.333333,1
78842,PMC1782267__imm0118-0010-f2.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,TGF–TAK1 pathway,"(NCBIGene:4088, NCBIGene:4089, NCBIGene:5872)","(NCBIGene:1386, NCBIGene:1432, NCBIGene:4087, ...","(NCBIGene:4088, NCBIGene:4089)",0.666667,RAB13,genetically_interacts_with,SMAD4,part_of,SMAD3,NCBIGene:5872,NCBIGene:5872,NCBIGene:4089,NCBIGene:4089,NCBIGene:4088,NCBIGene:4088,1195.333333,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85192,PMC6794807__12920_2019_596_Fig5_HTML.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,Protein-Protein interaction network and 5 iden...,"(MESH:D005047, NCBIGene:5872, NCBIGene:7316)","(MESH:C005084, MESH:C057561, MESH:C093415, MES...","(MESH:D005047, NCBIGene:5872)",0.666667,RAB13,genetically_interacts_with,UBC,resistance_associated_with,Etoposide,NCBIGene:5872,NCBIGene:5872,NCBIGene:7316,NCBIGene:7316,PUBCHEM.COMPOUND:36462,MESH:D005047,0.666667,144
48612,PMC4246236__viruses-06-04505-g001.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...,"Cholesterol metabolism, lipid rafts and cellul...","(MESH:D008623, NCBIGene:213, NCBIGene:5872)","(MESH:C006261, MESH:D000666, MESH:D002784, MES...","(NCBIGene:213, NCBIGene:5872)",0.666667,RAB13,entity_negatively_regulates_entity,ALB,entity_negatively_regulates_entity,beta-Mercaptoethanol,NCBIGene:5872,NCBIGene:5872,NCBIGene:213,NCBIGene:213,PUBCHEM.COMPOUND:1567,MESH:D008623,0.666667,780
14188,PMC3261702__msb201183-f4.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,Structural coverage of the core vision pathway...,"(MESH:C097613, NCBIGene:5147, NCBIGene:5872)","(MESH:C016077, MESH:C016276, MESH:C068152, MES...","(NCBIGene:5147, NCBIGene:5872)",0.666667,RAB13,genetically_interacts_with,PDE6D,resistance_associated_with,Imatinib,NCBIGene:5872,NCBIGene:5872,NCBIGene:5147,NCBIGene:5147,PUBCHEM.COMPOUND:5291,MESH:C097613,0.666667,138
14211,PMC3261702__msb201183-f4.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...

## Fisher's Exact Test

### TODO: vectorize
For better performance, try vectorizing the calculation of Fisher's Exact test, e.g.:
https://stackoverflow.com/questions/34947578/how-to-vectorize-fishers-exact-test

In [20]:
all_genes = set()
gene_instance_count = 0
all_chemicals = set()
chemical_instance_count = 0
all_diseases = set()
disease_instance_count = 0
for figure_id, pfocr_result in figure_id_to_pfocr_result.items():
    genes = pfocr_result["associatedWith"]["mentions"]["genes"]["ncbigene"]
    for gene in genes:
        gene_instance_count += 1
        all_genes.add(gene)
        
    chemicals = pfocr_result["associatedWith"]["mentions"]["chemicals"]["mesh"]
    for chemical in chemicals:
        chemical_instance_count += 1
        all_chemicals.add(chemical)
        
    diseases = pfocr_result["associatedWith"]["mentions"]["diseases"]["mesh"]
    for disease in diseases:
        disease_instance_count += 1
        all_diseases.add(disease)
    
print("total instance counts:")
print(gene_instance_count)
print(chemical_instance_count)
print(disease_instance_count)

print("")

print("unique counts:")
print(len(all_genes))
print(len(all_chemicals))
print(len(all_diseases))

print("")

curie_category_to_curies = {
    "biolink:ChemicalEntity": all_chemicals,
    "biolink:SmallMolecule": all_chemicals,
    "biolink:Disease": all_diseases,
    "biolink:Gene": all_genes,
}


all_figure_curies = set()
for curie_category in set(curie_category_to_curies.keys()).intersection(curie_categories):
    all_figure_curies |= curie_category_to_curies[curie_category]
all_figure_curie_count = len(all_figure_curies)

print(all_figure_curie_count)

total instance counts:
1369680
275456
20465

unique counts:
14253
14482
1430

28735


In [21]:
# trapi_curies_in_pwy
results_with_figures_df["yes_pwy_yes_trapi"] = (
    results_with_figures_df["overlap_curie_combo"].map(len)
)

# q_node_id_count - trapi_curies_in_pwy
results_with_figures_df["no_pwy_yes_trapi"] = (
    len(q_node_ids) - results_with_figures_df["yes_pwy_yes_trapi"]
)

# curies_in_pwy - trapi_curies_in_pwy
results_with_figures_df["yes_pwy_no_trapi"] = (
    results_with_figures_df["figure_curie_combo"].map(len) - 
    results_with_figures_df["yes_pwy_yes_trapi"]
)

# 28735 - curies_in_pwy - q_node_id_count - trapi_curies_in_pwy
results_with_figures_df["no_pwy_no_trapi"] = (
    all_figure_curie_count -
    results_with_figures_df["figure_curie_combo"].map(len) -
    len(q_node_ids) -
    results_with_figures_df["yes_pwy_yes_trapi"]
)

In [22]:
import scipy.stats as stats


results_with_figures_df["fishers_exact"] = results_with_figures_df[
    ["yes_pwy_yes_trapi",
     "no_pwy_yes_trapi",
     "yes_pwy_no_trapi",
     "no_pwy_no_trapi"]
].apply(
    lambda r: stats.fisher_exact([[
        r.yes_pwy_yes_trapi,
        r.no_pwy_yes_trapi
    ], [
        r.yes_pwy_no_trapi,
        r.no_pwy_no_trapi,
    ]]),
    axis=1)

results_with_figures_df["p_value"] = results_with_figures_df["fishers_exact"].apply(
    lambda x: x[1]
)

In [23]:
results_with_figures_df["p_value"].sort_values()

10080    2.001226e-08
10079    2.001226e-08
10025    2.159165e-08
10026    2.159165e-08
18611    2.180600e-08
             ...     
69868    3.067455e-04
78449    3.259109e-04
31216    4.788013e-04
4422     4.788013e-04
31215    4.788013e-04
Name: p_value, Length: 94380, dtype: float64

In [24]:
results_with_figures_df

,figure_id,figure_url,figure_title,trapi_result_curie_combo,figure_curie_combo,overlap_curie_combo,score,n0,e01,n1,...,n2_curie,n2_unified_curie,cumulative_trapi_result_curie_combo_figure_score,trapi_curie_combo_count_by_figure,yes_pwy_yes_trapi,no_pwy_yes_trapi,yes_pwy_no_trapi,no_pwy_no_trapi,fishers_exact,p_value
77770,PMC1998874__335_2007_9040_Fig4_HTML.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,Genes comprising simple linearly ordered pathw...,"(NCBIGene:4088, NCBIGene:4089, NCBIGene:5872)","(NCBIGene:4086, NCBIGene:4087, NCBIGene:4088, ...","(NCBIGene:4088, NCBIGene:4089)",0.666667,RAB13,genetically_interacts_with,SMAD4,...,NCBIGene:4088,NCBIGene:4088,1195.333333,1,2,1,8,28720,"(7180.0, 3.270369056034652e-07)",3.270369e-07
78846,PMC5765207__18_2017_2608_Fig3_HTML.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,Developmentally regulated signaling pathways i...,"(NCBIGene:4088, NCBIGene:4089, NCBIGene:5872)","(MESH:D014443, NCBIGene:10000, NCBIGene:11211,...","(NCBIGene:4088, NCBIGene:4089)",0.666667,RAB13,genetically_interacts_with,SMAD4,...,NCBIGene:4088,NCBIGene:4088,1195.333333,2,2,1,67,28661,"(855.5522388059702, 1.7026176901922824e-05)",1.702618e-05
78844,PMC8038106__ijms-22-03471-g001.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8...,Molecular pathophysiology of hereditary hemorr...,"(NCBIGene:4088, NCBIGene:4089, NCBIGene:5872)","(MESH:C064206, MESH:D011692, NCBIGene:2022, NC...","(NCBIGene:4088, NCBIGene:4089)",0.666667,RAB13,genetically_interacts_with,SMAD4,...,NCBIGene:4088,NCBIGene:4088,1195.333333,1,2,1,15,28713,"(3828.4, 9.882176241160208e-07)",9.882176e-07
78843,PMC548008__zmb0040547470007.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,The molecular mechanisms involved in the TGF-B...,"(NCBIGene:4088, NCBIGene:4089, NCBIGene:5872)","(NCBIGene:10000, NCBIGene:1432, NCBIGene:14685...","(NCBIGene:4088, NCBIGene:4089)",0.666667,RAB13,genetically_interacts_with,SMAD4,...,NCBIGene:4088,NCBIGene:4088,1195.333333,1,2,1,32,28696,"(1793.5, 4.074789037132147e-06)",4.074789e-06
78842,PMC1782267__imm0118-0010-f2.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,TGF–TAK1 pathway,"(NCBIGene:4088, NCBIGene:4089, NCBIGene:5872)","(NCBIGene:1386, NCBIGene:1432, NCBIGene:4087, ...","(NCBIGene:4088, NCBIGene:4089)",0.666667,RAB13,genetically_interacts_with,SMAD4,...,NCBIGene:4088,NCBIGene:4088,1195.333333,1,2,1,13,28715,"(4417.692307692308, 7.629975582113902e-07)",7.629976e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85192,PMC6794807__12920_2019_596_Fig5_HTML.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,Protein-Protein interaction network and 5 iden...,"(MESH:D005047, NCBIGene:5872, NCBIGene:7316)","(MESH:C005084, MESH:C057561, MESH:C093415, MES...","(MESH:D005047, NCBIGene:5872)",0.666667,RAB13,genetically_interacts_with,UBC,...,PUBCHEM.COMPOUND:36462,MESH:D005047,0.666667,144,2,1,124,28604,"(461.35483870967744, 5.70773730183569e-05)",5.707737e-05
48612,PMC4246236__viruses-06-04505-g001.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...,"Cholesterol metabolism, lipid rafts and cellul...","(MESH:D008623, NCBIGene:213, NCBIGene:5872)","(MESH:C006261, MESH:D000666, MESH:D002784, MES...","(NCBIGene:213, NCBIGene:5872)",0.666667,RAB13,entity_negatively_regulates_entity,ALB,...,PUBCHEM.COMPOUND:1567,MESH:D008623,0.666667,780,2,1,80,28648,"(716.2, 2.409499119451066e-05)",2.409499e-05
14188,PMC3261702__msb201183-f4.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,Structural coverage of the core vision pathway...,"(MESH:C097613, NCBIGene:5147, NCBIGene:5872)","(MESH:C016077, MESH:C016276, MESH:C068152, MES...","(NCBIGene:5147, NCBIGene:5872)",0.666667,RAB13,genetically_interacts_with,PDE6D,...,PUBCHEM.COMPOUND:5291,MESH:C097613,0.666667,138,2,1,97,28631,"(590.3298969072165, 3.518175444661935e-05)",3.518175e-05
14211,PMC3261702__msb201183-f4.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,Structural coverage of the core vision pathway...

## Analysis of results with vs. without BTE Sleeve

With the current query graph, requiring co-occurrence of at least 2 nodes in at least one PFOCR figure cuts the number of TRAPI result curie combos down to 887 from 3,738.

When requiring 3 nodes, it went down all the way to 0, but I've seen other queries where that yield 9 to 30 results.

Total number of curie combos returned:

In [25]:
len(results_df["trapi_result_curie_combo"].drop_duplicates())

5019

TRAPI result curie combos where at least `x` curies are also found in at least one figure:

In [26]:
for i in range(2, len(q_node_ids) + 1):
    curie_combo_count = len(set(
        trapi_figure_overlap_df[
            trapi_figure_overlap_df["overlap_curie_combo"].map(len) >= i
        ]["trapi_result_curie_combo"]
    ))
    print(f'when {i}+ overlapping curies: {curie_combo_count}')

when 2+ overlapping curies: 3516
when 3+ overlapping curies: 33


curie combos where curies for each pair of nodes are found in at least one figure:

In [27]:
from itertools import combinations


overlap_columns = results_with_figures_df["overlap_curie_combo"].apply(pd.Series)

for q_node_id in q_node_ids:
    overlap_df = results_with_figures_df[(
        results_with_figures_df[f"{q_node_id}_unified_curie"] == overlap_columns[0]
    ) | (
        results_with_figures_df[f"{q_node_id}_unified_curie"] == overlap_columns[1]
    )]
    print(f'{q_node_id}')
    print(f'  {len(overlap_df["trapi_result_curie_combo"].drop_duplicates())} TRAPI result(s)')
    print(f'  {len(overlap_df["figure_id"].drop_duplicates())} figure(s)')

for (a, b) in combinations(q_node_ids, 2):
    overlap_df = results_with_figures_df[((
        results_with_figures_df[f"{a}_unified_curie"] == overlap_columns[0]
    ) | (
        results_with_figures_df[f"{a}_unified_curie"] == overlap_columns[1]
    )) & ((
        results_with_figures_df[f"{b}_unified_curie"] == overlap_columns[0]
    ) | (
        results_with_figures_df[f"{b}_unified_curie"] == overlap_columns[1]
    ))]
    print(f'{a} & {b}')
    print(f'  {len(overlap_df["trapi_result_curie_combo"].drop_duplicates())} TRAPI result(s)')
    print(f'  {len(overlap_df["figure_id"].drop_duplicates())} figure(s)')

n0
  3499 TRAPI result(s)
  200 figure(s)
n1
  3395 TRAPI result(s)
  2901 figure(s)
n2
  681 TRAPI result(s)
  2816 figure(s)
n0 & n1
  3354 TRAPI result(s)
  195 figure(s)
n0 & n2
  390 TRAPI result(s)
  106 figure(s)
n1 & n2
  456 TRAPI result(s)
  2735 figure(s)


## View TRAPI results with figures

Note you can click the figures to go to the paper.

Compare these results with the `results_df` table earlier in this notebook and also with the [ARAX UI](https://arax.ncats.io/?r=44922) (click "Load" and then "Results".).

Does requiring co-occurrence for specific nodes like `n0` and `n2` help? Does ranking by p-value help?

### By lowest p-value

In [28]:
lowest_p_value_df = results_with_figures_df.sort_values(
    "p_value"
)

#### By TRAPI result

In [29]:
from IPython.display import Image
from IPython.core.display import HTML 


limit = 5
count = 0
display_count_limit = 10

for trapi_result_curie_combo, df1 in lowest_p_value_df.groupby("trapi_result_curie_combo", sort=False):
    trapi_curie_names = []
    for curie in df1["trapi_result_curie_combo"].tolist()[0]:
        name = curie_to_name[curie]
        trapi_curie_names.append(name)

    cumulative_trapi_result_curie_combo_figure_score = list(set(
        df1["cumulative_trapi_result_curie_combo_figure_score"]
    ))[0]
    
    ordered_trapi_result_curie_combo = []
    for q_node_id in q_node_ids:
        ordered_trapi_result_curie_combo.append(list(set(
            df1[f"{q_node_id}_unified_curie"]
        ))[0])

    display(HTML(data=f'''
<div style="border: 1px solid slategray; padding: 6px;">
  <span>{", ".join(ordered_trapi_result_curie_combo)}</span> 
  <span>(cumulative containment score: {cumulative_trapi_result_curie_combo_figure_score:.2f})</span>
</div>
'''))

    overlap_names = []
    for curie in df1["overlap_curie_combo"].tolist()[0]:
        name = curie_to_name[curie]
        overlap_names.append(name)

    trapi_result_rows = []
    for i,row in df1[trapi_result_columns].drop_duplicates().iterrows():
        trapi_result_chunks = []
        for trapi_result_column in trapi_result_columns:
            row_value = row[trapi_result_column]
            if row_value in overlap_names:
                trapi_result_chunk = f'''
<span style="border: 1px orange solid; padding: 1px; border-radius: 4px;">{row_value}</span>
'''
            else:
                trapi_result_chunk = f'{row_value}'
            trapi_result_chunks.append(trapi_result_chunk)
        trapi_result_rows.append(
            f'{" - ".join(trapi_result_chunks)}'
        )
    trapi_result_row_count = len(trapi_result_rows)
    display(HTML(data=f'''
<div style="font-size: small;">
{"<br>".join(trapi_result_rows)}
</div>
'''))

    figures = []
    row_height='100px'
    for figure_id, df0 in df1.groupby("figure_id", sort=False):
        [pmc,filename] = figure_id.split("__")
        paper_url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/{pmc}/"
        figure_url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/{pmc}/bin/{filename}"
        figure_title = list(set(df0["figure_title"]))[0]
        figure_title_limited = textwrap.shorten(figure_title, width=45, placeholder="...")

        p_value = list(set(
            df0["p_value"]
        ))[0]
        score = list(set(df0["score"]))[0]

        # TODO: why does the commented out block below not display correctly
        # on GitHub? The figcaption appears to the right of the image.
        # The div solution below is a kludge.
        figures.append(f'''
        <div style="margin: 5px !important; padding: 5px !important;>
            <figure style="margin: 5px !important;">
                <a target="_blank" href="{paper_url}">
                  <img src="{figure_url}" style="height: {row_height}">
              </a>
            </figure>
            <div style="font-size: x-small;">
                {figure_title_limited}<br>
                p-value: {p_value:.1e}, 
                containment: {score:.2}
            </div>
        </div>
        ''')
        
#        figures.append(f'''
#            <figure style="margin: 5px !important;">
#                <a target="_blank" href="{paper_url}">
#                  <img src="{figure_url}" style="height: {row_height}">
#                </a>
#                <figcaption style="font-size: small;">
#                      {figure_title_limited}<br>
#                      p-value: {p_value:.1e}, 
#                      containment: {score:.2}
#                </figcaption>
#            </figure>
#        ''')

    figure_count = len(figures)
    display(HTML(data=f'''
    <div style="display: flex; flex-flow: row wrap; text-align: center;">
    {''.join(figures[:display_count_limit])}
    </div>
'''))

    if figure_count > display_count_limit:
        print(f"...")
        print("")
        print(f"plus {figure_count - display_count_limit} more figures")
        print("")


    count += 1
    if count >= limit:
        print("...")
        print("")
        remaining = len(set(lowest_p_value_df["trapi_result_curie_combo"])) - limit
        print(f'plus {remaining} more TRAPI results')
        print("")
        break

...

plus 180 more figures



...

plus 9 more figures



...

plus 7 more figures



...

plus 105 more figures



...

plus 3511 more TRAPI results



#### By figure

In [30]:
from IPython.display import Image
from IPython.core.display import HTML 


limit = 3
count = 0
display_count_limit = 5

for figure_id, raw_df in lowest_p_value_df.groupby("figure_id", sort=False):
    df0 = raw_df.sort_values(
        "p_value"
    )
    [pmc,filename] = figure_id.split("__")
    paper_url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/{pmc}/"
    figure_url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/{pmc}/bin/{filename}"
    figure_title = list(set(df0["figure_title"]))[0]

    display(HTML(data=f'''
<div style="border: 1px solid slategray; padding: 6px; font-weight: bold;">
  {figure_title}
</div>
<figure style="margin: 5px !important;">
    <a target="_blank" href="{paper_url}">
      <img src="{figure_url}" style="max-height: 500px; max-width: 100%;">
    </a>
</figure>
'''))

    trapi_results_row_sets = []
    for trapi_result_curie_combo, df1 in df0[
        trapi_result_columns + ["trapi_result_curie_combo", "overlap_curie_combo", "p_value", "score"]
    ].drop_duplicates().groupby("trapi_result_curie_combo", sort=False):
        overlap_names = []
        for curie in df1["overlap_curie_combo"].tolist()[0]:
            name = curie_to_name[curie]
            overlap_names.append(name)

        trapi_results_rows = []
        for i,row in df1.iterrows():
            trapi_result_chunks = []
            for trapi_result_column in trapi_result_columns:
                row_value = row[trapi_result_column]
                if row_value in overlap_names:
                    trapi_result_chunk = f'''
<span style="border: 1px orange solid; padding: 1px; border-radius: 4px;">{row_value}</span>
'''
                else:
                    trapi_result_chunk = f'{row_value}'
                trapi_result_chunks.append(trapi_result_chunk)
                
            trapi_results_rows.append(
                f'{" - ".join(trapi_result_chunks)}</span>'
            )
            
        p_value = list(set(df1["p_value"]))[0]
        score = list(set(df1["score"]))[0]
        trapi_results_row_sets.append(f'''
<div>
    <span style="font-weight: bold; font-size: small;">
      TRAPI result p-value: {p_value:.1e}, containment: {score:.2}
    </span><br>
    {"<br>".join(trapi_results_rows)}
</div>
''')

    trapi_results_row_set_count = len(trapi_results_row_sets)
    display(HTML(data="<br>".join(trapi_results_row_sets[:display_count_limit])))
    if trapi_results_row_set_count > display_count_limit:
        print(f"...")
        print("")
        print(f"plus {trapi_results_row_set_count - display_count_limit} more TRAPI results")
    print("")

    count += 1
    if count >= limit:
        print("...")
        print("")
        print(f'plus {len(set(lowest_p_value_df["figure_id"])) - limit} more figures')
        print("")
        break

...

plus 136 more TRAPI results



...

plus 147 more TRAPI results




...

plus 2903 more figures



### co-occurrence: `n0` and (`n1` or `n2`)

In [31]:
n0_cooccurrence_df = results_with_figures_df[results_with_figures_df.apply(
    lambda r: r.n0_unified_curie in set(r.overlap_curie_combo),
    axis=1
)].sort_values(
    "p_value"
)

In [32]:
print(len(set(results_with_figures_df["figure_id"])))
print(len(set(n0_cooccurrence_df["figure_id"])))

2906
200


#### By TRAPI result

In [33]:
from IPython.display import Image
from IPython.core.display import HTML 


limit = 5
count = 0
display_count_limit = 10

for trapi_result_curie_combo, df1 in n0_cooccurrence_df.groupby("trapi_result_curie_combo", sort=False):
    trapi_curie_names = []
    for curie in df1["trapi_result_curie_combo"].tolist()[0]:
        name = curie_to_name[curie]
        trapi_curie_names.append(name)

    cumulative_trapi_result_curie_combo_figure_score = list(set(
        df1["cumulative_trapi_result_curie_combo_figure_score"]
    ))[0]
    
    ordered_trapi_result_curie_combo = []
    for q_node_id in q_node_ids:
        ordered_trapi_result_curie_combo.append(list(set(
            df1[f"{q_node_id}_unified_curie"]
        ))[0])

    display(HTML(data=f'''
<div style="border: 1px solid slategray; padding: 6px;">
  <span>{", ".join(ordered_trapi_result_curie_combo)}</span> 
  <span>(cumulative containment score: {cumulative_trapi_result_curie_combo_figure_score:.2f})</span>
</div>
'''))

    overlap_names = []
    for curie in df1["overlap_curie_combo"].tolist()[0]:
        name = curie_to_name[curie]
        overlap_names.append(name)

    trapi_result_rows = []
    for i,row in df1[trapi_result_columns].drop_duplicates().iterrows():
        trapi_result_chunks = []
        for trapi_result_column in trapi_result_columns:
            row_value = row[trapi_result_column]
            if row_value in overlap_names:
                trapi_result_chunk = f'''
<span style="border: 1px orange solid; padding: 1px; border-radius: 4px;">{row_value}</span>
'''
            else:
                trapi_result_chunk = f'{row_value}'
            trapi_result_chunks.append(trapi_result_chunk)
        trapi_result_rows.append(
            f'{" - ".join(trapi_result_chunks)}'
        )
    trapi_result_row_count = len(trapi_result_rows)
    display(HTML(data=f'''
<div style="font-size: small;">
{"<br>".join(trapi_result_rows)}
</div>
'''))

    figures = []
    row_height='100px'
    for figure_id, df0 in df1.groupby("figure_id", sort=False):
        [pmc,filename] = figure_id.split("__")
        paper_url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/{pmc}/"
        figure_url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/{pmc}/bin/{filename}"
        figure_title = list(set(df0["figure_title"]))[0]
        figure_title_limited = textwrap.shorten(figure_title, width=45, placeholder="...")

        p_value = list(set(
            df0["p_value"]
        ))[0]
        score = list(set(df0["score"]))[0]

        # TODO: why does the commented out block below not display correctly
        # on GitHub? The figcaption appears to the right of the image.
        # The div solution below is a kludge.
        figures.append(f'''
        <div style="margin: 5px !important; padding: 5px !important;>
            <figure style="margin: 5px !important;">
                <a target="_blank" href="{paper_url}">
                  <img src="{figure_url}" style="height: {row_height}">
              </a>
            </figure>
            <div style="font-size: x-small;">
                {figure_title_limited}<br>
                p-value: {p_value:.1e}, 
                containment: {score:.2}
            </div>
        </div>
        ''')
        
#        figures.append(f'''
#            <figure style="margin: 5px !important;">
#                <a target="_blank" href="{paper_url}">
#                  <img src="{figure_url}" style="height: {row_height}">
#                </a>
#                <figcaption style="font-size: small;">
#                      {figure_title_limited}<br>
#                      p-value: {p_value:.1e}, 
#                      containment: {score:.2}
#                </figcaption>
#            </figure>
#        ''')

    figure_count = len(figures)
    display(HTML(data=f'''
    <div style="display: flex; flex-flow: row wrap; text-align: center;">
    {''.join(figures[:display_count_limit])}
    </div>
'''))

    if figure_count > display_count_limit:
        print(f"...")
        print("")
        print(f"plus {figure_count - display_count_limit} more figures")
        print("")


    count += 1
    if count >= limit:
        print("...")
        print("")
        remaining = len(set(n0_cooccurrence_df["trapi_result_curie_combo"])) - limit
        print(f'plus {remaining} more TRAPI results')
        print("")
        break

...

plus 180 more figures



...

plus 3 more figures



...

plus 26 more figures



...

plus 3495 more TRAPI results



#### By figure

In [34]:
from IPython.display import Image
from IPython.core.display import HTML 


limit = 3
count = 0
display_count_limit = 5

for figure_id, raw_df in n0_cooccurrence_df.groupby("figure_id", sort=False):
    df0 = raw_df.sort_values(
        "p_value"
    )
    [pmc,filename] = figure_id.split("__")
    paper_url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/{pmc}/"
    figure_url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/{pmc}/bin/{filename}"
    figure_title = list(set(df0["figure_title"]))[0]

    display(HTML(data=f'''
<div style="border: 1px solid slategray; padding: 6px; font-weight: bold;">
  {figure_title}
</div>
<figure style="margin: 5px !important;">
    <a target="_blank" href="{paper_url}">
      <img src="{figure_url}" style="max-height: 500px; max-width: 100%;">
    </a>
</figure>
'''))

    trapi_results_row_sets = []
    for trapi_result_curie_combo, df1 in df0[
        trapi_result_columns + ["trapi_result_curie_combo", "overlap_curie_combo", "p_value", "score"]
    ].drop_duplicates().groupby("trapi_result_curie_combo", sort=False):
        overlap_names = []
        for curie in df1["overlap_curie_combo"].tolist()[0]:
            name = curie_to_name[curie]
            overlap_names.append(name)

        trapi_results_rows = []
        for i,row in df1.iterrows():
            trapi_result_chunks = []
            for trapi_result_column in trapi_result_columns:
                row_value = row[trapi_result_column]
                if row_value in overlap_names:
                    trapi_result_chunk = f'''
<span style="border: 1px orange solid; padding: 1px; border-radius: 4px;">{row_value}</span>
'''
                else:
                    trapi_result_chunk = f'{row_value}'
                trapi_result_chunks.append(trapi_result_chunk)
                
            trapi_results_rows.append(
                f'{" - ".join(trapi_result_chunks)}</span>'
            )
            
        p_value = list(set(df1["p_value"]))[0]
        score = list(set(df1["score"]))[0]
        trapi_results_row_sets.append(f'''
<div>
    <span style="font-weight: bold; font-size: small;">
      TRAPI result p-value: {p_value:.1e}, containment: {score:.2}
    </span><br>
    {"<br>".join(trapi_results_rows)}
</div>
''')

    trapi_results_row_set_count = len(trapi_results_row_sets)
    display(HTML(data="<br>".join(trapi_results_row_sets[:display_count_limit])))
    if trapi_results_row_set_count > display_count_limit:
        print(f"...")
        print("")
        print(f"plus {trapi_results_row_set_count - display_count_limit} more TRAPI results")
    print("")

    count += 1
    if count >= limit:
        print("...")
        print("")
        print(f'plus {len(set(n0_cooccurrence_df["figure_id"])) - limit} more figures')
        print("")
        break

...

plus 136 more TRAPI results



...

plus 147 more TRAPI results



...

plus 775 more TRAPI results

...

plus 197 more figures



### co-occurrence: `n0` and `n1`

In [35]:
n0_n1_cooccurrence_df = results_with_figures_df[results_with_figures_df.apply(
    lambda r: (
        r.n0_unified_curie in set(r.overlap_curie_combo)
    ) and (
        r.n1_unified_curie in set(r.overlap_curie_combo)
    ),
    axis=1
)].sort_values(
    "p_value"
)

#### By TRAPI result

In [36]:
from IPython.display import Image
from IPython.core.display import HTML 


limit = 5
count = 0
display_count_limit = 10

for trapi_result_curie_combo, df1 in n0_n1_cooccurrence_df.groupby("trapi_result_curie_combo", sort=False):
    trapi_curie_names = []
    for curie in df1["trapi_result_curie_combo"].tolist()[0]:
        name = curie_to_name[curie]
        trapi_curie_names.append(name)

    cumulative_trapi_result_curie_combo_figure_score = list(set(
        df1["cumulative_trapi_result_curie_combo_figure_score"]
    ))[0]
    
    ordered_trapi_result_curie_combo = []
    for q_node_id in q_node_ids:
        ordered_trapi_result_curie_combo.append(list(set(
            df1[f"{q_node_id}_unified_curie"]
        ))[0])

    display(HTML(data=f'''
<div style="border: 1px solid slategray; padding: 6px;">
  <span>{", ".join(ordered_trapi_result_curie_combo)}</span> 
  <span>(cumulative containment score: {cumulative_trapi_result_curie_combo_figure_score:.2f})</span>
</div>
'''))

    overlap_names = []
    for curie in df1["overlap_curie_combo"].tolist()[0]:
        name = curie_to_name[curie]
        overlap_names.append(name)

    trapi_result_rows = []
    for i,row in df1[trapi_result_columns].drop_duplicates().iterrows():
        trapi_result_chunks = []
        for trapi_result_column in trapi_result_columns:
            row_value = row[trapi_result_column]
            if row_value in overlap_names:
                trapi_result_chunk = f'''
<span style="border: 1px orange solid; padding: 1px; border-radius: 4px;">{row_value}</span>
'''
            else:
                trapi_result_chunk = f'{row_value}'
            trapi_result_chunks.append(trapi_result_chunk)
        trapi_result_rows.append(
            f'{" - ".join(trapi_result_chunks)}'
        )
    trapi_result_row_count = len(trapi_result_rows)
    display(HTML(data=f'''
<div style="font-size: small;">
{"<br>".join(trapi_result_rows)}
</div>
'''))

    figures = []
    row_height='100px'
    for figure_id, df0 in df1.groupby("figure_id", sort=False):
        [pmc,filename] = figure_id.split("__")
        paper_url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/{pmc}/"
        figure_url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/{pmc}/bin/{filename}"
        figure_title = list(set(df0["figure_title"]))[0]
        figure_title_limited = textwrap.shorten(figure_title, width=45, placeholder="...")

        p_value = list(set(
            df0["p_value"]
        ))[0]
        score = list(set(df0["score"]))[0]

        # TODO: why does the commented out block below not display correctly
        # on GitHub? The figcaption appears to the right of the image.
        # The div solution below is a kludge.
        figures.append(f'''
        <div style="margin: 5px !important; padding: 5px !important;>
            <figure style="margin: 5px !important;">
                <a target="_blank" href="{paper_url}">
                  <img src="{figure_url}" style="height: {row_height}">
              </a>
            </figure>
            <div style="font-size: x-small;">
                {figure_title_limited}<br>
                p-value: {p_value:.1e}, 
                containment: {score:.2}
            </div>
        </div>
        ''')
        
#        figures.append(f'''
#            <figure style="margin: 5px !important;">
#                <a target="_blank" href="{paper_url}">
#                  <img src="{figure_url}" style="height: {row_height}">
#                </a>
#                <figcaption style="font-size: small;">
#                      {figure_title_limited}<br>
#                      p-value: {p_value:.1e}, 
#                      containment: {score:.2}
#                </figcaption>
#            </figure>
#        ''')

    figure_count = len(figures)
    display(HTML(data=f'''
    <div style="display: flex; flex-flow: row wrap; text-align: center;">
    {''.join(figures[:display_count_limit])}
    </div>
'''))

    if figure_count > display_count_limit:
        print(f"...")
        print("")
        print(f"plus {figure_count - display_count_limit} more figures")
        print("")


    count += 1
    if count >= limit:
        print("...")
        print("")
        remaining = len(set(n0_n1_cooccurrence_df["trapi_result_curie_combo"])) - limit
        print(f'plus {remaining} more TRAPI results')
        print("")
        break

...

plus 180 more figures



...

plus 3352 more TRAPI results



#### By figure

In [37]:
from IPython.display import Image
from IPython.core.display import HTML 


limit = 3
count = 0
display_count_limit = 5

for figure_id, raw_df in n0_n1_cooccurrence_df.groupby("figure_id", sort=False):
    df0 = raw_df.sort_values(
        "p_value"
    )
    [pmc,filename] = figure_id.split("__")
    paper_url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/{pmc}/"
    figure_url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/{pmc}/bin/{filename}"
    figure_title = list(set(df0["figure_title"]))[0]

    display(HTML(data=f'''
<div style="border: 1px solid slategray; padding: 6px; font-weight: bold;">
  {figure_title}
</div>
<figure style="margin: 5px !important;">
    <a target="_blank" href="{paper_url}">
      <img src="{figure_url}" style="max-height: 500px; max-width: 100%;">
    </a>
</figure>
'''))

    trapi_results_row_sets = []
    for trapi_result_curie_combo, df1 in df0[
        trapi_result_columns + ["trapi_result_curie_combo", "overlap_curie_combo", "p_value", "score"]
    ].drop_duplicates().groupby("trapi_result_curie_combo", sort=False):
        overlap_names = []
        for curie in df1["overlap_curie_combo"].tolist()[0]:
            name = curie_to_name[curie]
            overlap_names.append(name)

        trapi_results_rows = []
        for i,row in df1.iterrows():
            trapi_result_chunks = []
            for trapi_result_column in trapi_result_columns:
                row_value = row[trapi_result_column]
                if row_value in overlap_names:
                    trapi_result_chunk = f'''
<span style="border: 1px orange solid; padding: 1px; border-radius: 4px;">{row_value}</span>
'''
                else:
                    trapi_result_chunk = f'{row_value}'
                trapi_result_chunks.append(trapi_result_chunk)
                
            trapi_results_rows.append(
                f'{" - ".join(trapi_result_chunks)}</span>'
            )
            
        p_value = list(set(df1["p_value"]))[0]
        score = list(set(df1["score"]))[0]
        trapi_results_row_sets.append(f'''
<div>
    <span style="font-weight: bold; font-size: small;">
      TRAPI result p-value: {p_value:.1e}, containment: {score:.2}
    </span><br>
    {"<br>".join(trapi_results_rows)}
</div>
''')

    trapi_results_row_set_count = len(trapi_results_row_sets)
    display(HTML(data="<br>".join(trapi_results_row_sets[:display_count_limit])))
    if trapi_results_row_set_count > display_count_limit:
        print(f"...")
        print("")
        print(f"plus {trapi_results_row_set_count - display_count_limit} more TRAPI results")
    print("")

    count += 1
    if count >= limit:
        print("...")
        print("")
        print(f'plus {len(set(n0_n1_cooccurrence_df["figure_id"])) - limit} more figures')
        print("")
        break

...

plus 126 more TRAPI results



...

plus 126 more TRAPI results



...

plus 767 more TRAPI results

...

plus 192 more figures



### co-occurrence: `n0` and `n2`

In [38]:
n0_n2_cooccurrence_df = results_with_figures_df[results_with_figures_df.apply(
    lambda r: (
        r.n0_unified_curie in set(r.overlap_curie_combo)
    ) and (
        r.n2_unified_curie in set(r.overlap_curie_combo)
    ),
    axis=1
)].sort_values(
    "p_value"
)

#### By TRAPI result

In [39]:
from IPython.display import Image
from IPython.core.display import HTML 


limit = 5
count = 0
display_count_limit = 10

for trapi_result_curie_combo, df1 in n0_n2_cooccurrence_df.groupby("trapi_result_curie_combo", sort=False):
    trapi_curie_names = []
    for curie in df1["trapi_result_curie_combo"].tolist()[0]:
        name = curie_to_name[curie]
        trapi_curie_names.append(name)

    cumulative_trapi_result_curie_combo_figure_score = list(set(
        df1["cumulative_trapi_result_curie_combo_figure_score"]
    ))[0]
    
    ordered_trapi_result_curie_combo = []
    for q_node_id in q_node_ids:
        ordered_trapi_result_curie_combo.append(list(set(
            df1[f"{q_node_id}_unified_curie"]
        ))[0])

    display(HTML(data=f'''
<div style="border: 1px solid slategray; padding: 6px;">
  <span>{", ".join(ordered_trapi_result_curie_combo)}</span> 
  <span>(cumulative containment score: {cumulative_trapi_result_curie_combo_figure_score:.2f})</span>
</div>
'''))

    overlap_names = []
    for curie in df1["overlap_curie_combo"].tolist()[0]:
        name = curie_to_name[curie]
        overlap_names.append(name)

    trapi_result_rows = []
    for i,row in df1[trapi_result_columns].drop_duplicates().iterrows():
        trapi_result_chunks = []
        for trapi_result_column in trapi_result_columns:
            row_value = row[trapi_result_column]
            if row_value in overlap_names:
                trapi_result_chunk = f'''
<span style="border: 1px orange solid; padding: 1px; border-radius: 4px;">{row_value}</span>
'''
            else:
                trapi_result_chunk = f'{row_value}'
            trapi_result_chunks.append(trapi_result_chunk)
        trapi_result_rows.append(
            f'{" - ".join(trapi_result_chunks)}'
        )
    trapi_result_row_count = len(trapi_result_rows)
    display(HTML(data=f'''
<div style="font-size: small;">
{"<br>".join(trapi_result_rows)}
</div>
'''))

    figures = []
    row_height='100px'
    for figure_id, df0 in df1.groupby("figure_id", sort=False):
        [pmc,filename] = figure_id.split("__")
        paper_url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/{pmc}/"
        figure_url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/{pmc}/bin/{filename}"
        figure_title = list(set(df0["figure_title"]))[0]
        figure_title_limited = textwrap.shorten(figure_title, width=45, placeholder="...")

        p_value = list(set(
            df0["p_value"]
        ))[0]
        score = list(set(df0["score"]))[0]

        # TODO: why does the commented out block below not display correctly
        # on GitHub? The figcaption appears to the right of the image.
        # The div solution below is a kludge.
        figures.append(f'''
        <div style="margin: 5px !important; padding: 5px !important;>
            <figure style="margin: 5px !important;">
                <a target="_blank" href="{paper_url}">
                    <img src="{figure_url}" style="height: {row_height}">
                </a>
            </figure>
            <div style="font-size: x-small;">
                {figure_title_limited}<br>
                p-value: {p_value:.1e}, 
                containment: {score:.2}
            </div>
        </div>
        ''')
        
#        figures.append(f'''
#            <figure style="margin: 5px !important;">
#                <a target="_blank" href="{paper_url}">
#                  <img src="{figure_url}" style="height: {row_height}">
#                </a>
#                <figcaption style="font-size: small;">
#                      {figure_title_limited}<br>
#                      p-value: {p_value:.1e}, 
#                      containment: {score:.2}
#                </figcaption>
#            </figure>
#        ''')

    figure_count = len(figures)
    display(HTML(data=f'''
    <div style="display: flex; flex-flow: row wrap; text-align: center;">
    {''.join(figures[:display_count_limit])}
    </div>
'''))

    if figure_count > display_count_limit:
        print(f"...")
        print("")
        print(f"plus {figure_count - display_count_limit} more figures")
        print("")


    count += 1
    if count >= limit:
        print("...")
        print("")
        remaining = len(set(n0_n2_cooccurrence_df["trapi_result_curie_combo"])) - limit
        print(f'plus {remaining} more TRAPI results')
        print("")
        break

...

plus 3 more figures



...

plus 3 more figures



...

plus 26 more figures

...

plus 404 more TRAPI results



#### By figure

In [40]:
from IPython.display import Image
from IPython.core.display import HTML 


limit = 3
count = 0
display_count_limit = 5

for figure_id, raw_df in n0_n2_cooccurrence_df.groupby("figure_id", sort=False):
    df0 = raw_df.sort_values(
        "p_value"
    )
    [pmc,filename] = figure_id.split("__")
    paper_url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/{pmc}/"
    figure_url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/{pmc}/bin/{filename}"
    figure_title = list(set(df0["figure_title"]))[0]

    display(HTML(data=f'''
<div style="border: 1px solid slategray; padding: 6px; font-weight: bold;">
  {figure_title}
</div>
<figure style="margin: 5px !important;">
    <a target="_blank" href="{paper_url}">
      <img src="{figure_url}" style="max-height: 500px; max-width: 100%;">
    </a>
</figure>
'''))

    trapi_results_row_sets = []
    for trapi_result_curie_combo, df1 in df0[
        trapi_result_columns + ["trapi_result_curie_combo", "overlap_curie_combo", "p_value", "score"]
    ].drop_duplicates().groupby("trapi_result_curie_combo", sort=False):
        overlap_names = []
        for curie in df1["overlap_curie_combo"].tolist()[0]:
            name = curie_to_name[curie]
            overlap_names.append(name)

        trapi_results_rows = []
        for i,row in df1.iterrows():
            trapi_result_chunks = []
            for trapi_result_column in trapi_result_columns:
                row_value = row[trapi_result_column]
                if row_value in overlap_names:
                    trapi_result_chunk = f'''
<span style="border: 1px orange solid; padding: 1px; border-radius: 4px;">{row_value}</span>
'''
                else:
                    trapi_result_chunk = f'{row_value}'
                trapi_result_chunks.append(trapi_result_chunk)
                
            trapi_results_rows.append(
                f'{" - ".join(trapi_result_chunks)}</span>'
            )
            
        p_value = list(set(df1["p_value"]))[0]
        score = list(set(df1["score"]))[0]
        trapi_results_row_sets.append(f'''
<div>
    <span style="font-weight: bold; font-size: small;">
      TRAPI result p-value: {p_value:.1e}, containment: {score:.2}
    </span><br>
    {"<br>".join(trapi_results_rows)}
</div>
''')

    trapi_results_row_set_count = len(trapi_results_row_sets)
    display(HTML(data="<br>".join(trapi_results_row_sets[:display_count_limit])))
    if trapi_results_row_set_count > display_count_limit:
        print(f"...")
        print("")
        print(f"plus {trapi_results_row_set_count - display_count_limit} more TRAPI results")
    print("")

    count += 1
    if count >= limit:
        print("...")
        print("")
        print(f'plus {len(set(n0_n2_cooccurrence_df["figure_id"])) - limit} more figures')
        print("")
        break

...

plus 6 more TRAPI results



...

plus 17 more TRAPI results



...

plus 7 more TRAPI results

...

plus 107 more figures

